In [121]:
import os
import sys
import imp
import pprint

sys.path.append('/nuwa_cluster/home/jackyu/climatools/')
import climatools.misc as climamisc
imp.reload(climamisc)

from IPython.display import HTML

In [122]:
def related_fortran_files():
    dir_rrtmg = '/nuwa_cluster/home/jackyu/climate_models/cesm1_2_2/\
models/atm/cam/src/physics/rrtmg/'
    
    relative_filepaths = ['radlw.F90',
                         'ext/rrtmg_mcica/rrtmg_lw_rad.f90',
                          'ext/rrtmg_mcica/mcica_subcol_gen_lw.f90'
                         ]
    
    absolute_paths = [os.path.join(dir_rrtmg, filepath) for filepath in relative_filepaths]
    return absolute_paths

In [123]:
def print_subroutines(fpath):
    subroutines = read_subroutines_from_file(fpath)
    for subroutine in subroutines:
        print(climamisc.get_Fortran_subroutine_name(subroutine))
        for called_subroutine in sorted(list(climamisc.get_called_Fortran_subroutine_names(subroutine))):
            print('    ', called_subroutine)
        print()


In [124]:
pprint.pprint(climamisc.Fortran_subroutine_relations_from_files(paths_fortran = related_fortran_files()))

{'broadcast_along_gpoints': {'childs': [],
                             'parents': ['cloud_overlap_lw_clirad_maxrandom']},
 'cast_against_iscloudy': {'childs': [],
                           'parents': ['cloud_overlap_lw_clirad_maxrandom']},
 'cldscale': {'childs': [], 'parents': ['cloud_overlap_lw_clirad_maxrandom']},
 'cloud_overlap_lw_clirad_maxrandom': {'childs': ['cast_against_iscloudy',
                                                  'lmhovrlp_probabilities',
                                                  'broadcast_along_gpoints',
                                                  'get_iscloudy',
                                                  'setcoef_sw',
                                                  'cldscale'],
                                       'parents': ['rad_rrtmg_lw']},
 'generate_stochastic_clouds': {'childs': ['endrun', 'kissvec'],
                                'parents': ['mcica_subcol_lw']},
 'get_iscloudy': {'childs': [],
                  'parents'

Quick scan of cloud variables through these subroutines suggests that the flow is more or less the same as shortwave.  It appears that optical depth, $\tau$, is the only optical property passed along.